In [ ]:
import os

try:
    if CONFIGURED_DIRS:  # type: ignore
        pass
except:
    PLAYGROUND_DIR = os.getcwd()
    os.chdir("../src")

CONFIGURED_DIRS = True

print(f"Playground directory: {PLAYGROUND_DIR}")  # type: ignore
print(f"Working directory: {os.getcwd()}")

In [ ]:
import sys
import importlib
from IPython.display import Audio

from common.roll import Roll, RollExportConfig
from common.structures.note import Note
from common.structures.pitch import Pitch

from generation.instruments.voice import Voice
from generation.instruments.piano import Piano
from generation.instruments.bass_drum import BassDrum
from generation.instruments.snare_drum import SnareDrum

In [ ]:
sys.path.append("../")  # allow modules one directory above to be accessible

In [ ]:
def create_song(name: str) -> Roll:
    song = importlib.import_module(f"playground.songs.{name}")
    roll, melody, chords = song.get_song()

    voice = roll.add_instrument("Voice", Voice)
    assert type(voice) is Voice
    piano = roll.add_instrument("Stride Piano", Piano)
    assert type(piano) is Piano
    bass_drum = roll.add_instrument("Bass Drum", BassDrum)
    assert type(bass_drum) is BassDrum
    snare_drum = roll.add_instrument("Snare Drum", SnareDrum)
    assert type(snare_drum) is SnareDrum

    final_pitch = chords.chords[-1].chord.root
    final_pitch = final_pitch.reoctave_near_pitch(Pitch.from_str("C3"))
    voice.generate(melody)
    piano.generate(chords)
    bass_drum.generate(melody)
    snare_drum.generate(melody)
    piano.notes.add(Note(final_pitch, chords.end_time, roll.Duration(1)))

    return roll


def export_song(name: str) -> str:
    output_path = os.path.join(PLAYGROUND_DIR, "output", f"{name}.wav")
    create_song(name).export(RollExportConfig(output_path=output_path))
    return output_path

In [ ]:
Audio(export_song("first"))

In [ ]:
Audio(export_song("happy_birthday"))